In [3]:
import torch
import torch.nn as nn

In [4]:
class Conv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super(Conv, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, stride, padding)

    def forward(self, x):
        return self.conv2(self.conv1(x))

In [5]:
in_channels = 3
out_channels = 3
kernel_size = 3
stride = 1
padding = 1

input = torch.arange(1, 17, dtype=torch.float32)
input = input.reshape(4, 4)
input = torch.stack((input, input, input), dim=0)

input = input.unsqueeze(0)

print(input.shape)

torch.Size([1, 3, 4, 4])


In [6]:
conv_layer = Conv(in_channels, out_channels, kernel_size, stride, padding)

output = conv_layer(input)

print(output.shape)

torch.Size([1, 3, 4, 4])


In [7]:
print(output[0][0])

tensor([[ 1.5348,  1.6115, -0.4391, -0.4817],
        [ 2.0207,  0.8244, -1.5717, -1.7923],
        [ 0.6763, -2.3158, -3.5350, -3.7699],
        [-0.4406, -3.7272, -2.7415, -3.7501]], grad_fn=<SelectBackward0>)


In [8]:
print(output[0][1])

tensor([[-3.4580, -3.6071, -1.8558, -0.5881],
        [-4.8117, -5.2596, -1.9670, -0.8374],
        [-2.6026, -3.3094,  0.4499, -0.0967],
        [ 0.3516, -0.6049,  2.2788,  0.0111]], grad_fn=<SelectBackward0>)


In [22]:
conv_layer.conv2.bias

Parameter containing:
tensor([-0.1249, -0.0846,  0.1488], requires_grad=True)

In [24]:
conv_layer.conv1.bias

Parameter containing:
tensor([-0.0797, -0.0452, -0.0243], requires_grad=True)

In [11]:
mod = torch.jit.trace(conv_layer, input)

In [12]:
mod.save("conv_layer.pt")

In [13]:
print(input)

tensor([[[[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])


In [16]:
conv_layer.conv1.weight.shape

torch.Size([3, 3, 3, 3])

torch.Size([3, 3, 3])

In [21]:
print(conv_layer.conv1.weight[0][0].transpose(0, 1))

tensor([[ 0.1282,  0.0085, -0.1548],
        [ 0.1363,  0.1247, -0.0382],
        [-0.1844,  0.0565,  0.0695]], grad_fn=<TransposeBackward0>)


In [26]:
conv_layer.conv1.bias

Parameter containing:
tensor([-0.0797, -0.0452, -0.0243], requires_grad=True)

In [29]:
loaded_model = torch.jit.load("/home/roger/infer/ncnn/tools/pnnx/build/install/bin/conv_layer.pt")  

In [31]:
input

tensor([[[[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [34]:
loaded_model.conv1.bias

tensor([ 0.1479, -0.0939, -0.1641], requires_grad=True)

In [ ]:
loaded_model.conv1.

In [32]:
output = loaded_model(input)

In [33]:
output

tensor([[[[ 0.4478,  0.1848, -0.2786, -0.0372],
          [ 0.0603, -0.8329, -1.5097, -0.8818],
          [-1.4466, -2.4896, -1.5121, -0.8260],
          [-0.1719, -1.7644, -1.7853, -2.0658]],

         [[-0.5090, -0.8490, -0.6545, -0.4924],
          [-0.7347, -0.6911, -0.8197, -0.6600],
          [-2.9306, -1.2793, -1.7790,  0.5636],
          [ 0.9264,  1.8373,  1.2763, -0.0607]],

         [[ 0.9913,  0.9891,  1.6925,  0.3219],
          [ 1.4916,  1.7119,  2.3004,  0.4872],
          [ 1.2896,  0.8821,  0.5474, -1.0274],
          [ 0.1304,  2.1233, -0.1227,  0.9236]]]],
       grad_fn=<ConvolutionBackward1>)